In [3]:
import yaml
with open('tree.yaml') as f:
    # use safe_load instead load
    dataMap = yaml.safe_load(f)
    print(dataMap)

with open('newtree.yaml', "w") as f:
    yaml.dump(dataMap, f)

{'treeroot': {'branch1': {'name': 'Node 1', 'branch1-1': {'name': 'Node 1-1'}}, 'branch2': {'name': 'Node 2', 'branch2-1': {'name': 'Node 2-1'}}}}


In [14]:
print(dataMap['treeroot']['branch1'])
root=dataMap['treeroot']
for k, branch in root.items():
    print(k, branch['name'])

{'name': 'Node 1', 'branch1-1': {'name': 'Node 1-1'}}
branch1 Node 1
branch2 Node 2


In [1]:
import sys
sys.path.append("..")
from sagas.util.str_converters import to_camel_case

In [74]:
to_camel_case("people_line", True)

'PeopleLine'

In [86]:
def is_true(node, attr):
    return (attr in node) and node[attr]
def is_primary(node):
    return is_true(node, "primary")

class ModelDefinitions(object):
    def __init__(self, file_name):
        self.models={}
        with open(file_name) as f:
            # use safe_load instead load
            doc = yaml.safe_load(f)
            self.models_root=doc['models']
            self.database_root=doc['database']
            for k, model in self.models_root.items():        
                print(k, model['name'])
                if is_true(model, 'abstract'):
                    print("\tabstract", model['abstract'])
                else:
                    self.models[k]=model

model_def=ModelDefinitions('model_planet.yaml')
print(".. getting people fields")
for k, fld in model_def.models['people']['fields'].items():
    print(k, is_primary(fld), is_true(fld, 'input'))

db=model_def.database_root
print(db['url'])

base_model Base model
	abstract True
people People
planet Planet
.. getting people fields
id True False
name False False
height False False
mass False False
hair_color False False
skin_color False False
eye_color False False
birth_year False False
gender False False
planet_id False False
url False False
cockroachdb://maxroach@localhost:26257/bank


In [81]:
model_header='''
from .base import Base
from sqlalchemy import Column, ForeignKey, Integer, String


class Model{cname}(Base):
    """{cname} model."""

    __tablename__ = '{name}'
'''

# model_name='people'
model_name='planet'
print(model_header.format(name=model_name, 
                          cname=to_camel_case(model_name, True),
                          model=model_def.models[model_name]))


from .base import Base
from sqlalchemy import Column, ForeignKey, Integer, String


class ModelPlanet(Base):
    """Planet model."""

    __tablename__ = 'planet'



In [88]:
field_def='''{fld_name} = Column('{fld_name}', {fld_type}, {pk_part} doc="{field[doc]}")'''
rel_def='''{lname} = relationship(Model{model}, backref='{field[backref]}')'''
field_builder=[]
for k, fld in model_def.models[model_name]['fields'].items():
    primary_part=""
    if is_primary(fld):
        primary_part="primary_key=True,"
    if fld['type']=="relation":
        field_builder.append(rel_def.format(
            lname=to_camel_case(k),
            model=to_camel_case(fld['model'], True),
            field=fld
            ))
    else:
        # print(fld['doc'])
        field_builder.append(field_def.format(fld_name=k, 
                           fld_type=fld['type'].capitalize(),
                           pk_part=primary_part,
                           field=fld))

print("    "+"\n    ".join(field_builder))

    id = Column('id', Integer, primary_key=True, doc="Id of the planet")
    name = Column('name', String,  doc="Name of the planet.")
    rotation_period = Column('rotation_period', String,  doc="Rotation period of the planet.")
    orbital_period = Column('orbital_period', String,  doc="Orbital period of the planet.")
    diameter = Column('diameter', String,  doc="Diameter of the planet.")
    climate = Column('climate', String,  doc="Climate period of the planet.")
    gravity = Column('gravity', String,  doc="Gravity of the planet.")
    terrain = Column('terrain', String,  doc="Terrain of the planet.")
    surface_water = Column('surface_water', String,  doc="Surface water of the planet.")
    population = Column('population', String,  doc="Population of the planet.")
    url = Column('url', String,  doc="URL of the planet in the Star Wars API.")
    peopleList = relationship(ModelPeople, backref='planet')


In [89]:
schema_header='''from datetime import datetime
from graphene_sqlalchemy import SQLAlchemyObjectType
from database.base import db_session
from database.model_{name} import Model{cname}
import graphene
import utils


# Create a generic class to mutualize description of {name} attributes for both queries and mutations
class {cname}Attribute:
'''

print(schema_header.format(name=model_name, 
                           cname=to_camel_case(model_name, True),
                           model=model_def.models[model_name]))

from datetime import datetime
from graphene_sqlalchemy import SQLAlchemyObjectType
from database.base import db_session
from database.model_planet import ModelPlanet
import graphene
import utils


# Create a generic class to mutualize description of planet attributes for both queries and mutations
class PlanetAttribute:



In [90]:
type_map={"string":"String", "integer":"Int"}

def is_input(node):
    if 'input' in node:
        return node['input']
    return True
def get_graphql_type(fld_name, fld_type):
    if fld_name.endswith('_id'):
        return "ID"
    return type_map[fld_type]

field_def='''{fld_name} = graphene.{fld_type}(description="{field[doc]}")'''
field_builder=[]
for k, fld in model_def.models[model_name]['fields'].items():
    if fld['type']=="relation":
        pass
    elif is_input(fld):        
        field_builder.append(field_def.format(fld_name=k, 
                           fld_type=get_graphql_type(k,fld['type']),
                           field=fld))

print("    "+"\n    ".join(field_builder))

    name = graphene.String(description="Name of the planet.")
    rotation_period = graphene.String(description="Rotation period of the planet.")
    orbital_period = graphene.String(description="Orbital period of the planet.")
    diameter = graphene.String(description="Diameter of the planet.")
    climate = graphene.String(description="Climate period of the planet.")
    gravity = graphene.String(description="Gravity of the planet.")
    terrain = graphene.String(description="Terrain of the planet.")
    surface_water = graphene.String(description="Surface water of the planet.")
    population = graphene.String(description="Population of the planet.")
    url = graphene.String(description="URL of the planet in the Star Wars API.")


In [91]:
creator_def='''
class {model[name]}(SQLAlchemyObjectType):
    """{model[name]} node."""

    class Meta:
        model = Model{model[name]}
        interfaces = (graphene.relay.Node,)

###
class Create{model[name]}Input(graphene.InputObjectType, {model[name]}Attribute):
    """Arguments to create a {name}."""
    pass


class Create{model[name]}(graphene.Mutation):
    """Mutation to create a {name}."""
    {name} = graphene.Field(lambda: {model[name]}, description="{model[name]} created by this mutation.")

    class Arguments:
        input = Create{model[name]}Input(required=True)

    def mutate(self, info, input):
        data = utils.input_to_dictionary(input)
        data['created'] = str(datetime.utcnow())
        data['edited'] = str(datetime.utcnow())

        {name} = Model{model[name]}(**data)
        db_session.add({name})
        db_session.commit()

        return Create{model[name]}({name}={name})
'''
print(creator_def.format(name=model_name, 
                         cname=to_camel_case(model_name, True),
                         model=model_def.models[model_name]))


class Planet(SQLAlchemyObjectType):
    """Planet node."""

    class Meta:
        model = ModelPlanet
        interfaces = (graphene.relay.Node,)

###
class CreatePlanetInput(graphene.InputObjectType, PlanetAttribute):
    """Arguments to create a planet."""
    pass


class CreatePlanet(graphene.Mutation):
    """Mutation to create a planet."""
    planet = graphene.Field(lambda: Planet, description="Planet created by this mutation.")

    class Arguments:
        input = CreatePlanetInput(required=True)

    def mutate(self, info, input):
        data = utils.input_to_dictionary(input)
        data['created'] = str(datetime.utcnow())
        data['edited'] = str(datetime.utcnow())

        planet = ModelPlanet(**data)
        db_session.add(planet)
        db_session.commit()

        return CreatePlanet(planet=planet)



In [92]:
updater_def='''
class Update{model[name]}Input(graphene.InputObjectType, {model[name]}Attribute):
    """Arguments to update a {name}."""
    id = graphene.ID(required=True, description="Global Id of the {name}.")


class Update{model[name]}(graphene.Mutation):
    """Update a {name}."""
    {name} = graphene.Field(lambda: {model[name]}, description="{model[name]} updated by this mutation.")

    class Arguments:
        input = Update{model[name]}Input(required=True)

    def mutate(self, info, input):
        data = utils.input_to_dictionary(input)
        data['edited'] = str(datetime.utcnow())

        {name} = db_session.query(Model{model[name]}).filter_by(id=data['id'])
        {name}.update(data)
        db_session.commit()
        {name} = db_session.query(Model{model[name]}).filter_by(id=data['id']).first()

        return Update{model[name]}({name}={name})
'''
print(updater_def.format(name=model_name, 
                         cname=to_camel_case(model_name, True),
                         model=model_def.models[model_name]))


class UpdatePlanetInput(graphene.InputObjectType, PlanetAttribute):
    """Arguments to update a planet."""
    id = graphene.ID(required=True, description="Global Id of the planet.")


class UpdatePlanet(graphene.Mutation):
    """Update a planet."""
    planet = graphene.Field(lambda: Planet, description="Planet updated by this mutation.")

    class Arguments:
        input = UpdatePlanetInput(required=True)

    def mutate(self, info, input):
        data = utils.input_to_dictionary(input)
        data['edited'] = str(datetime.utcnow())

        planet = db_session.query(ModelPlanet).filter_by(id=data['id'])
        planet.update(data)
        db_session.commit()
        planet = db_session.query(ModelPlanet).filter_by(id=data['id']).first()

        return UpdatePlanet(planet=planet)



In [93]:
## schema.py
lines=[]
lines.append('''from graphene_sqlalchemy import SQLAlchemyConnectionField
import graphene''')
for mo in model_def.models:
    lines.append("import schema_{}".format(mo))

## queries
lines.append('''
class Query(graphene.ObjectType):
    """Nodes which can be queried by this API."""
    node = graphene.relay.Node.Field()''')
for mo in model_def.models:
    lines.append('''    # {cname}
    {name} = graphene.relay.Node.Field(schema_{name}.{cname})
    {name}List = SQLAlchemyConnectionField(schema_{name}.{cname})'''.format(
        name=mo, cname=mo.capitalize()))
## mutations
lines.append('''
class Mutation(graphene.ObjectType):
    """Mutations which can be performed by this API."""''')
for mo in model_def.models:
    lines.append('''    # {cname} mutation
    create{cname} = schema_{name}.Create{cname}.Field()
    update{cname} = schema_{name}.Update{cname}.Field()'''.format(
        name=mo, cname=mo.capitalize()))

lines.append("schema = graphene.Schema(query=Query, mutation=Mutation)")
print("\n".join(lines))

from graphene_sqlalchemy import SQLAlchemyConnectionField
import graphene
import schema_people
import schema_planet

class Query(graphene.ObjectType):
    """Nodes which can be queried by this API."""
    node = graphene.relay.Node.Field()
    # People
    people = graphene.relay.Node.Field(schema_people.People)
    peopleList = SQLAlchemyConnectionField(schema_people.People)
    # Planet
    planet = graphene.relay.Node.Field(schema_planet.Planet)
    planetList = SQLAlchemyConnectionField(schema_planet.Planet)

class Mutation(graphene.ObjectType):
    """Mutations which can be performed by this API."""
    # People mutation
    createPeople = schema_people.CreatePeople.Field()
    updatePeople = schema_people.UpdatePeople.Field()
    # Planet mutation
    createPlanet = schema_planet.CreatePlanet.Field()
    updatePlanet = schema_planet.UpdatePlanet.Field()
schema = graphene.Schema(query=Query, mutation=Mutation)


In [69]:
## setup.py
lines=[]
lines.append('''from ast import literal_eval
from database import base
import logging
import sys''')
for mo in model_def.models:
    lines.append("from database.model_{} import Model{}".format(mo, mo.capitalize()))
lines.append('''
# Load logging configuration
log = logging.getLogger(__name__)
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


if __name__ == '__main__':
    #+ remove all tables
    base.Base.metadata.drop_all(base.engine)

    log.info('Create database {}'.format(base.db_name))
    base.Base.metadata.create_all(base.engine)''')    
for mo in model_def.models:
    lines.append('''    log.info('Insert {cname} data in database')
    with open('database/data/{name}.json', 'r') as file:
        data = literal_eval(file.read())
        for record in data:
            {name} = Model{cname}(**record)
            base.db_session.add({name})
        base.db_session.commit()'''.format(
        name=mo, cname=mo.capitalize()))

print("\n".join(lines))    

from ast import literal_eval
from database import base
import logging
import sys
from database.model_people import ModelPeople
from database.model_planet import ModelPlanet

# Load logging configuration
log = logging.getLogger(__name__)
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


if __name__ == '__main__':
    #+ remove all tables
    base.Base.metadata.drop_all(base.engine)

    log.info('Create database {}'.format(base.db_name))
    base.Base.metadata.create_all(base.engine)
    log.info('Insert People data in database')
    with open('database/data/people.json', 'r') as file:
        data = literal_eval(file.read())
        for record in data:
            people = ModelPeople(**record)
            base.db_session.add(people)
        base.db_session.commit()
    log.info('Insert Planet data in database')
    with open('database/data/planet.json', 'r') as file:
        data = literal_eval(file